In [23]:
import ROOT as r
import numpy as np 
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)
def Unpack_Histo(histogram_file):
    bin_content = []
    bin_content_uncer =[]
    for i in range(1,histogram_file.GetNbinsX()+1):
        bin_content.append(histogram_file.GetBinContent(i))
        bin_content_uncer.append(histogram_file.GetBinError(i))
        
    return np.array(bin_content),np.array(bin_content_uncer)

############# CONFIGURING THE JOB ###########################
samples_path = "/home/diego/HEP/VBF/TauLep/"
sample1_name = "Ztautau_Sherpa"
sample2_name = "Ztautau_MG"



In [24]:
r.TH1.AddDirectory(r.kFALSE)
file1 = r.TFile.Open(samples_path+sample1_name+'.root')
file2 = r.TFile.Open(samples_path+sample2_name+'.root')

myFile =r.TFile.Open("Average.root", "RECREATE")

histos1 = []
histos2 = []

for i in file1.GetListOfKeys():
    histos1.append(str(i.GetName()))
    
for i in file2.GetListOfKeys():
    histos2.append(str(i.GetName()))
    
    
for histo in histos1 :
    
    if histo=="mass_jj":
    
        if histo in histos2:

            bin_range = [(file1.Get(histo)).GetXaxis().GetXmin(),file1.Get(histo).GetXaxis().GetXmax()]

            content1, uncer1 = Unpack_Histo(file1.Get(histo).Rebin(50))
            content2, uncer2 = Unpack_Histo(file2.Get(histo).Rebin(50))

            new_content = []
            new_uncer = []

            for i in range(0,len(content1)):
                if content1[i]!=0 and content2[i]!=0:
                    new_content.append((content1[i]/uncer1[i]**2+content2[i]/uncer2[i]**2)/(1/uncer1[i]**2+1/uncer2[i]**2))
                    new_uncer.append(1/(1/uncer1[i]**2+1/uncer2[i]**2))
                elif content1[i]!=0:
                    new_content.append(content2[i])
                    new_uncer.append(uncer2[i])
                elif content2[i]!=0:
                    new_content.append(content1[i])
                    new_uncer.append(uncer1[i])
                else :
                    new_content.append(0.0)
                    new_uncer.append(0.0)

            r.TH1.AddDirectory(r.kFALSE)
            final_hist = r.TH1F(histo,histo,len(new_content),bin_range[0],bin_range[1])

            for i in range(1,final_hist.GetNbinsX()+1):

                final_hist.SetBinContent(i,new_content[i-1])
                final_hist.SetBinError(i,new_uncer[i-1])

            myFile.WriteObject(final_hist,histo)

myFile.Close()

In [25]:

print(content1)


[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.61306357e+00 1.81559181e+01 3.35912094e+01 4.88206291e+01
 5.46771164e+01 4.77617989e+01 4.04087029e+01 4.09236145e+01
 3.25116577e+01 2.91105213e+01 2.37594490e+01 1.90360851e+01
 2.12926941e+01 1.49669847e+01 1.65707626e+01 1.35473185e+01
 8.70192432e+00 9.95579052e+00 8.03633022e+00 5.93532133e+00
 4.86358976e+00 4.23136234e+00 3.97852254e+00 2.35694408e+00
 2.31049681e+00 2.14221525e+00 1.75707197e+00 2.23770905e+00
 1.86860943e+00 2.40016317e+00 1.03567159e+00 8.33868861e-01
 8.04032087e-01 7.08849490e-01 5.33422589e-01 5.09694874e-01
 5.81120193e-01 3.96917284e-01 4.56649303e-01 2.56646663e-01
 3.14318001e-01 1.82123277e-02 1.81234539e-01 9.43045765e-02
 1.25293836e-01 1.53622717e-01 7.36540034e-02 1.58055425e-01
 1.02223627e-01 3.75888683e-02 9.86025333e-02 2.45991163e-02
 1.72666311e-02 2.15110909e-02 4.42183651e-02 8.87200050e-03]
